Подгрузка необходимых библиотек

In [62]:
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from numba import cuda

Функция поиска подстрок на CPU

In [63]:
def mass_search_CPU(N: np.ndarray, H: np.ndarray, R: np.ndarray):
  for j in range(R.shape[1]):
    for i in range(R.shape[0]):
      n = N[i]
      for k in range(len(n)):
        if n[k] == H[j]:
          R[i, j - k] -= 1
  return R

Функция поиска подстрок на GPU

In [64]:
@cuda.jit
def mass_search_GPU(N: np.ndarray, H: np.ndarray, R: np.ndarray):
	# Получение асболютного значения индекса потока в сетке
	x, y = cuda.grid(2)
	# Рассматриваем каждый символ подстроки
	n = N[y]
	for k in range(len(n)):
		# Декремент, если символ подстроки совпал с символом входного буффера
		if n[k] == H[x]:
			R[y, x - k] -= 1
	cuda.syncthreads()

Запись входных данных (N и H) в файл

In [65]:
def save_input(pathfile: str, N: np.ndarray, H: np.ndarray, mode='a', sheet_name=''):

  # Приведение списка символов (подстроки) к типу str
  # для корректной записи в excel
  new_N = [str(n) for n in N]
  
  # Создание DataFrame
  df = pd.DataFrame({"N": new_N, "H": H})
  
  # Если стоит mode='a', то первая запись в файл выдаст ошибку,
  # поэтому в этом случае mode меняется на 'w'.
  try:
    with pd.ExcelWriter(pathfile, mode=mode) as writer:
      df.to_excel(writer, sheet_name=sheet_name)
  except:
    with pd.ExcelWriter(pathfile, mode='w') as writer:
      df.to_excel(writer, sheet_name=sheet_name)

Запись рабочей матрицы R в файл

In [66]:
def save_R(pathfile: str, result: np.ndarray, mode='a'):

  df = pd.DataFrame(result)

  sheet_name = str(len(N))
  
  try:
    with pd.ExcelWriter(pathfile, mode=mode) as writer:
      df.to_excel(writer, sheet_name=sheet_name)
  except:
    with pd.ExcelWriter(pathfile, mode='w') as writer:
      df.to_excel(writer, sheet_name=sheet_name)


Вычисления на CPU и GPU i раз для усреднения времени

In [67]:
sizes = np.arange(160, 1600, 160)  # Размерности N и H
i = 8                            # Кол-во итераций для подсчёта среднего времени
N_chars = 2                       # Кол-во символов в подстроках
BLOCK_DIM = 32                    # Размерность оси квадратного блока
BLOCK_SIZE = (BLOCK_DIM, BLOCK_DIM)

alp = np.arange(256)

df = {"Время на GPU":np.zeros(len(sizes), dtype=float), 
      "Время на CPU":np.zeros(len(sizes), dtype=float),
      "Ускорения":np.zeros(len(sizes), dtype=float),
      "Одинаковые ли значения?":np.full(len(sizes), fill_value=True)}

df = pd.DataFrame(df, index=sizes)

# Вычисления для разных размерностей N и H
for size in sizes:

  # Случайная генерация подстрок N и входного буффера H
  N = np.random.randint(len(alp), size=(size, N_chars), dtype=np.uint8)
  H = np.random.randint(len(alp), size=size, dtype=np.uint8)

  # Заполнение рабочей матрицы NxH значениями N_chars
  R = np.full((size, size), fill_value=N_chars)
  # R = np.zeros((size, size), dtype = int)

  # Инициализация сетки блоков
  GRID_SIZE = ((len(N) + BLOCK_DIM - 1) // BLOCK_DIM , (len(H) + BLOCK_DIM - 1) // BLOCK_DIM)

  # Суммы общего времени за определённое кол-во итераций для реализации
  gpu_common_time = []
  cpu_common_time = []

  # Выполнение одних и тех же вычислений для нахождения среднего времени
  for _ in range (1):

    #Вычисление времени на GPU
    start_time = time()
    
    # Пересылка данных на устройство
    dev_N = cuda.to_device(N)
    dev_H = cuda.to_device(H)
    dev_R = cuda.to_device(R)
    
    # Запуск вычислений на GPU
    mass_search_GPU[GRID_SIZE, BLOCK_SIZE](dev_N, dev_H, dev_R)

    # Копирование рабочей матрицы с устройства на хост
    host_R = dev_R.copy_to_host()
    gpu_common_time.append(time() - start_time)

    #Вычисление времени на CPU
    start_time = time()
    R = mass_search_CPU(N, H, R.copy())
    cpu_common_time.append(time() - start_time)

    #if _ == 0:
    save_R("R_GPU.xlsx", host_R)
    save_R("R_CPU.xlsx", R)
  
  # Запись подсчётов времени и соответствия рабочих матриц
  df.loc[size, "Время на GPU"] = np.mean(gpu_common_time[0:])
  df.loc[size, "Время на CPU"] = np.mean(cpu_common_time[0:])
  df.loc[size, "Одинаковые ли значения?"] = np.array_equal(host_R, R)


  # Сохранение входных данных в файл с несколькими листами
  save_input("input_data.xlsx", N, H, sheet_name=f'Размер {size}')

# Запись значений ускорения GPU над CPU
df["Ускорение"] = df["Время на CPU"] / df["Время на GPU"]
df

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 25 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 100 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


,Время на GPU,Время на CPU,Ускорения,Одинаковые ли значения?,Ускорение
160,0.162479,0.019348,0.0,True,0.119078
320,0.002857,0.085074,0.0,True,29.777685
480,0.002756,0.182420,0.0,True,66.181645
640,0.003428,0.319964,0.0,True,93.345482
800,0.004474,0.667060,0.0,True,149.099547
960,0.006517,0.733069,0.0,True,112.490908
1120,0.006926,0.994946,0.0,True,143.652565
1280,0.010491,1.275810,0.0,True,121.613986
1440,0.010882,1.616858,0.0,True,148.575782
